In [1]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import time
import bookquery
import sys
import os
import datetime
from tqdm import tqdm_notebook

ua = UserAgent()#Прокси

def counter_for_files(path):
    """
    Подсчет файлов в директории
    """
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

def execute_query(qquery, entity_id , property_id):# Работа с запросами
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id , property_id = property_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparql(results):
    
    """Обработка запроса"""
    
    res = []
    for result in results:# Обработка вернувшегося запрсов
        entities = {}
        entities['qualifier_id'] = result['unit']['value'].split('/')[-1]
        entities['qualifier'] = result['unitLabel']['value']
        entities['value'] = 1
    return entities#Возвращаем Датафрейм и название сущности

In [2]:
def processing(entity_id , qualifier , f = 0):
    if f>5:
        print(entity_id)
        return None
    try:
        resu = entities_sparql(execute_query(bookquery.qualifier_finder, entity_id , qualifier))
        return resu
    except UnboundLocalError as e:
        return None
    except Exception as e:
        time.sleep(3)
        return processing(entity_id,qualifier, f+1)
    
    
def file_processing(file , qualifier):
    result =[]
    SIZE = 1000
    df = pd.read_csv(file , encoding='utf8',sep = ';')
    for k,v in  df[:SIZE].iterrows():
        result.append(dask.delayed(processing)(v['entity'] , qualifier))
    data = list(filter(None , dask.compute(*result)))
    results = pd.DataFrame(data = data)
    return results

In [4]:
%%time
predicates = {'численность населения':'население'}

# qualifier = 'P2046'

def qualifier_finder(predicate,predicate_name):
    path = join(os.getcwd() , r'Information\2\data\data_with_popularity')
    dff = pd.read_csv(r'csv\Properties.csv' , encoding='utf-8-sig',sep=';')
    for k,v in dff.iterrows():
        if v['label'].lower() == predicate.lower():
            qualifier = v['property']
            break
    main_df = pd.DataFrame()
    for file in tqdm_notebook(counter_for_files(path) , desc = 'files'):
        df = file_processing(file , qualifier)
        main_df = pd.concat([main_df,df])
    count = len(main_df)
    main_df = main_df.groupby(['qualifier_id','qualifier']).sum().sort_values(by = 'value',ascending = False)
    main_df.reset_index(inplace = True)
    main_df = main_df[['qualifier' , 'qualifier_id' , 'value']]
    main_df['value , %'] = round(main_df['value']/count * 100 , 2)
    try:
        with pd.ExcelWriter(r'csv\Qualifiers.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            main_df.to_excel(writer, sheet_name = f'{predicate_name}({qualifier})' , encoding='utf-8-sig' , index = False)
    except FileNotFoundError:
        main_df.to_excel(r'csv\Qualifiers.xlsx', sheet_name = f'{predicate_name}({qualifier})'
                         ,encoding='utf-8-sig' , index = False)
        
for k,v in tqdm_notebook(predicates.items() , desc = 'predicates'):
    qualifier_finder(k,v)



Wall time: 16min 54s


In [ ]:
'длина':'длина' ,
              'ширина': 'ширина', 
              'высота над уровнем моря': 'высота над уровнем моря',
              'относительная высота' : 'относительная высота',
              f'высота/рост':'высота,рост',
              'вертикальная глубина': 'глубина' ,
              'площадь': 'площадь' ,